In [1]:
import  torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.utils.data as td
import torch.nn.functional as F

定义变量

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_works = 4
batch_size = 64
input_dim=100
epochs = 1
print("run on ",device.type)

run on  cpu


In [3]:
from typing import Iterable


# loading data
def loading_minist_data(batch_siz:int, works:int)->(Iterable,Iterable):

    train_set = torchvision.datasets.MNIST(root="./data",train=True,transform=torchvision.transforms.ToTensor(),download=True)
    test_set = torchvision.datasets.MNIST(root="./data",train=False,transform=torchvision.transforms.ToTensor(),download=True)
    return td.DataLoader(dataset=train_set, batch_size=batch_siz, shuffle=True, num_workers=works,drop_last=True), td.DataLoader(dataset=test_set, batch_size=batch_siz, shuffle=True, num_workers=works,drop_last=True)


In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            # 28*28 -> (28 )/2 = 12*12
            # nn.Conv2d(1, 32,kernel_size=3,padding=1, stride=2,dtype=torch.float32),
            # nn.LeakyReLU(0.01),
            # 14,14 -> 7*7
            # nn.Conv2d(32,64,kernel_size=3,padding=1,stride=2,dtype=torch.float32),
            # nn.BatchNorm2d(64),
            # nn.LeakyReLU(0.01),
            # 10 filters to 10 filters
            # 7*7 -> 3*3
            # nn.Conv2d(64, 128, kernel_size=3,stride=2,dtype=torch.float32),
            # nn.LeakyReLU(0.02),
            # nn.BatchNorm2d(128),
            nn.Flatten(),
            # nn.Linear(128*3*3,10),
            nn.Linear(28*28,128),
            nn.ReLU(),
            nn.Linear(128,10),
            nn.Sigmoid()
        ).to(device)
        self.cnt=0
        self.loss_metric=[]
        self.loss_func = nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.parameters())
        pass
    def forward(self,inputs):
        for i in self.model:
            # print(inputs.shape)
            inputs = i(inputs)
        return inputs
        # return self.model(inputs)
    def train(self,inputs,targets):
        out = self.forward(inputs)

        loss = self.loss_func(out,targets)
        self.cnt+=1
        self.loss_metric.append(loss.item())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        pass

class Generator(nn.Module):
    def __init__(self,in_dim:int):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(in_dim, 200),
            nn.LeakyReLU(0.02),
            nn.LayerNorm(200),
            nn.Linear(200, 784),
            nn.Sigmoid()
        ).to(device)
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)
        # counter and accumulator for progress
        self.cnt = 0
        self.loss_metric = []


    def forward(self,inputs):
        return self.model(inputs).view(batch_size,28,28)
    def train(self,D:Discriminator,inputs,targets):
        gen_out = self.forward(inputs)
        # print(gen_out.shape)
        d_out = D.forward(gen_out)

        loss  = D.loss_func(d_out,targets)
                # zero gradients, perform a backward pass, update weights
        self.cnt+=1
        self.loss_metric.append(loss.item())
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()

def generate_random_seed(size):
    random_data = torch.randn(size*batch_size).view(batch_size,size)
    return random_data.to(device)


In [5]:
%%time
G = Generator(input_dim)
D = Discriminator()
train_iter ,test_iter = loading_minist_data(batch_size,4)
print("generate nums is run on ",generate_random_seed(input_dim).device)

for k in range(epochs):

    for it,y in train_iter:
    # print(y)
        out = D.forward(it)
    # y = torch.reshape(y,out.shape)
        loss = F.cross_entropy(out,y)
        # print(loss.item())
        # print(loss.sum())
    # print(loss)
        D.optimizer.zero_grad()
        loss.backward()
        D.optimizer.step()
        # break
    # break
# output = G.forward(generate_random_seed(input_dim))
# img = output.detach().numpy().reshape(28,28)
# plt.imshow(output.cpu().detach().numpy()[0], interpolation='none', cmap='Blues')

generate nums is run on  cpu
CPU times: user 32.9 s, sys: 5.97 s, total: 38.8 s
Wall time: 7.86 s


In [6]:
loss_metric = []
cnt=0
for it,y in test_iter:
    out = D.forward(it)
    loss = F.cross_entropy(out,y)
    cnt+=1
    if(cnt%10==0):
        loss_metric.append(loss.item())


In [31]:
print(loss_metric)
output = G.forward(generate_random_seed(input_dim))
img = output.reshape(64,1,28,28)
img0 = img[0]
print(img.shape)
boxes = torch.tensor([[135, 37, 160, 198]], dtype=torch.int8)
img0=torchvision.utils.draw_bounding_boxes(image=img0,boxes=boxes,labels="img0")
# torchvision.utils.save_image(img,"imgs/%d.png" % 0)
# plt.imshow(img0, interpolation='none', cmap='Blues')

[1.547363042831421, 1.523353934288025, 1.5411396026611328, 1.555657148361206, 1.5328975915908813, 1.5116971731185913, 1.5261415243148804, 1.5566644668579102, 1.54177725315094, 1.5050500631332397, 1.535434603691101, 1.5451445579528809, 1.5747623443603516, 1.5445125102996826, 1.5558316707611084]
torch.Size([64, 1, 28, 28])


ValueError: Tensor uint8 expected, got torch.float32